In [1]:
import json
import os

# Load all JSON files saved in Week 3
city_files = [
    'new_york_restaurants_foursquare.json',
    'san_francisco_restaurants_foursquare.json',
    'chicago_restaurants_foursquare.json',
    'boston_restaurants_foursquare.json',
    'los_angeles_restaurants_foursquare.json'
]

all_data = []

for file in city_files:
    with open(file, 'r') as f:
        data = json.load(f)
        all_data.extend(data)

print(f"✅ Loaded {len(all_data)} total restaurant records.")


✅ Loaded 994 total restaurant records.


In [5]:
# Save to combined JSON
with open('foursquare_all_restaurants_combined.json', 'w') as f:
    json.dump(all_data, f, indent=2)

print("✅ Saved to foursquare_all_restaurants_combined.json")


✅ Saved to foursquare_all_restaurants_combined.json


In [7]:
import pandas as pd

# Normalize and convert to DataFrame
df = pd.json_normalize(all_data)

# Save to CSV
df.to_csv('foursquare_all_restaurants_combined.csv', index=False)

print("✅ Saved to foursquare_all_restaurants_combined.csv")


✅ Saved to foursquare_all_restaurants_combined.csv


In [15]:
import json
import pandas as pd
import sqlite3

# Step 1: Load JSON
with open('foursquare_all_restaurants_combined.json', 'r') as f:
    data = json.load(f)

# Step 2: Normalize
df = pd.json_normalize(data)

# Step 3: Fix list-type columns (flatten to comma-separated strings)
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        df[col] = df[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

# Step 4: Save to SQLite
conn = sqlite3.connect('foursquare_restaurants.db')
df.to_sql(name='restaurants', con=conn, if_exists='replace', index=False)
conn.close()

print("✅ Successfully saved to foursquare_restaurants.db (table: restaurants)")




✅ Successfully saved to foursquare_restaurants.db (table: restaurants)


In [2]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('foursquare_restaurants.db')

# View first few rows
df = pd.read_sql_query("SELECT * FROM restaurants LIMIT 5;", conn)
conn.close()

df.head()

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.drop_off.latitude,geocodes.drop_off.longitude,...,location.locality,location.postcode,location.region,location.address_extended,geocodes.front_door.latitude,geocodes.front_door.longitude,related_places.parent.fsq_id,related_places.parent.categories,related_places.parent.name,related_places.children
0,5d5f24ec09484500079aee00,"{'id': 13306, 'name': 'Taco Restaurant', 'shor...",,LikelyOpen,283,/v3/places/5d5f24ec09484500079aee00,Los Tacos No. 1,America/New_York,40.71416,-74.008566,...,New York,10007,NY,None,NaN,NaN,None,None,None,None
1,574456e5498e39c73d234da1,"{'id': 13034, 'name': 'Café', 'short_name': 'C...",,Unsure,715,/v3/places/574456e5498e39c73d234da1,Black Fox Coffee,America/New_York,40.70631,-74.007883,...,New York,10005,NY,None,NaN,NaN,None,None,None,None
2,3fd66200f964a52083e61ee3,"{'id': 13068, 'name': 'American Restaurant', '...",,LikelyOpen,498,/v3/places/3fd66200f964a52083e61ee3,The Odeon,America/New_York,NaN,NaN,...,New York,10013,NY,Frnt A,NaN,NaN,None,None,None,None
3,4ea0afbf9adf1e334e4cc0e6,"{'id': 13034, 'name': 'Café', 'short_name': 'C...",,Unsure,616,/v3/places/4ea0afbf9adf1e334e4cc0e6,Laughing Man Coffee & Tea,America/New_York,NaN,NaN,...,New York,10013,NY,None,NaN,NaN,None,None,None,None
4,50bcb93ae4b0efcb4c84d434,"{'id': 13009, 'name': 'Cocktail Bar', 'short_n...",,LikelyOpen,1149,/v3/places/50bcb93ae4b0efcb4c84d434,The Dead Rabbit Grocery & Grog,America/New_York,NaN,NaN,...,New York,10004,NY,None,40.703225,-74.010998,None,None,None,None


In [4]:
import json
import os
import pandas as pd
import sqlite3

# === Load and merge all Week 3 JSON files ===
city_files = [
    'new_york_restaurants_foursquare.json',
    'san_francisco_restaurants_foursquare.json',
    'chicago_restaurants_foursquare.json',
    'boston_restaurants_foursquare.json',
    'los_angeles_restaurants_foursquare.json'
]

all_data = []

for file in city_files:
    with open(file, 'r') as f:
        data = json.load(f)
        all_data.extend(data)

print(f"✅ Loaded {len(all_data)} total restaurant records.")

# === Save to combined JSON ===
with open('foursquare_all_restaurants_combined.json', 'w') as f:
    json.dump(all_data, f, indent=2)

print("✅ Saved to foursquare_all_restaurants_combined.json")

# === Convert to DataFrame ===
df = pd.json_normalize(all_data)

# === Ensure 'price' column exists ===
if 'price' not in df.columns:
    df['price'] = None  # Add column if missing entirely

# === Fix list-type columns ===
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        df[col] = df[col].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)

# === Save to CSV ===
df.to_csv('foursquare_all_restaurants_combined.csv', index=False)
print("✅ Saved to foursquare_all_restaurants_combined.csv")

# === Save to SQLite ===
# Clean up column names for SQLite compatibility
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace(r'\W+', '_', regex=True)
df.columns = df.columns.str.replace('__+', '_', regex=True)
df.columns = df.columns.str.lower()

conn = sqlite3.connect('foursquare_restaurants.db')
df.to_sql(name='restaurants', con=conn, if_exists='replace', index=False)
conn.close()
print("✅ Saved to SQLite: foursquare_restaurants.db (table: restaurants)")


✅ Loaded 994 total restaurant records.
✅ Saved to foursquare_all_restaurants_combined.json
✅ Saved to foursquare_all_restaurants_combined.csv
✅ Saved to SQLite: foursquare_restaurants.db (table: restaurants)


In [6]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect('foursquare_restaurants.db')

# View first few rows
df = pd.read_sql_query("SELECT * FROM restaurants LIMIT 5;", conn)
conn.close()

df.head()

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes_drop_off_latitude,geocodes_drop_off_longitude,...,location_postcode,location_region,location_address_extended,geocodes_front_door_latitude,geocodes_front_door_longitude,related_places_parent_fsq_id,related_places_parent_categories,related_places_parent_name,related_places_children,price
0,5d5f24ec09484500079aee00,"{'id': 13306, 'name': 'Taco Restaurant', 'shor...",,LikelyOpen,283,/v3/places/5d5f24ec09484500079aee00,Los Tacos No. 1,America/New_York,40.71416,-74.008566,...,10007,NY,None,NaN,NaN,None,None,None,None,None
1,574456e5498e39c73d234da1,"{'id': 13034, 'name': 'Café', 'short_name': 'C...",,Unsure,715,/v3/places/574456e5498e39c73d234da1,Black Fox Coffee,America/New_York,40.70631,-74.007883,...,10005,NY,None,NaN,NaN,None,None,None,None,None
2,3fd66200f964a52083e61ee3,"{'id': 13068, 'name': 'American Restaurant', '...",,LikelyOpen,498,/v3/places/3fd66200f964a52083e61ee3,The Odeon,America/New_York,NaN,NaN,...,10013,NY,Frnt A,NaN,NaN,None,None,None,None,None
3,4ea0afbf9adf1e334e4cc0e6,"{'id': 13034, 'name': 'Café', 'short_name': 'C...",,Unsure,616,/v3/places/4ea0afbf9adf1e334e4cc0e6,Laughing Man Coffee & Tea,America/New_York,NaN,NaN,...,10013,NY,None,NaN,NaN,None,None,None,None,None
4,50bcb93ae4b0efcb4c84d434,"{'id': 13009, 'name': 'Cocktail Bar', 'short_n...",,LikelyOpen,1149,/v3/places/50bcb93ae4b0efcb4c84d434,The Dead Rabbit Grocery & Grog,America/New_York,NaN,NaN,...,10004,NY,None,40.703225,-74.010998,None,None,None,None,None
